In [1]:
import os
import re
import docx2txt
import pandas as pd
import tkinter as tk
from tkinter import filedialog

Esse código só funciona com arquivos .docx e que não estão vazios (0 kbytes). Baseado nos Recibos da Campinas Notebook Celulares

In [2]:
def find_pattern(pattern,text):
    target = re.compile(pattern)
    finded = target.findall(text)
    if len(finded) < 1:
        finded = ""
    return finded

def find_largepattern(pattern,text):
    match = re.search(pattern, text, re.DOTALL)
    if match:
        captured_text = match.group(1)
    return captured_text.strip()

def select_folder():
    root = tk.Tk()
    root.withdraw()
    folder_path = filedialog.askdirectory()
    if folder_path:
        return folder_path


In [3]:
# dataframe vazio
full_data = pd.DataFrame(columns=['Arquivo', 'Recibo', 'Ordem de Servico',
                                  'Data', 'Cliente', 'CPF/CNPJ', 'Telefone',
                                 'Email', 'Endereco', 'Descrição'])

In [ ]:
folder_path = select_folder()
os.chdir(folder_path)

for file in os.listdir():
    if file.endswith('.docx') == True:
        text = docx2txt.process(file)
        
        # Nome do arquivo
        arquivo = str(file)

        # recibo
        pattern = r'RECIBO #\s?(.+)'
        recibo = find_pattern(pattern, text)
   
        # oss
        pattern = r'E-Mail\n\n(.+)'
        oss = find_pattern(pattern, text)

        # data
        pattern = r'Data:\s?(.+)\n'
        data_entrada = find_pattern(pattern, text)

        # nome_cliente
        pattern = r'Comprador\(a\):\n\n(.+)'
        nome_cliente = find_pattern(pattern, text)
        
        # cpf_cnpj
        pattern = r"CPF/CNPJ:\s*(.*)\s*RG"
        cpf_cnpj = find_pattern(pattern, text)
        cpf_cnpj[0] = cpf_cnpj[0].replace(" ", "").replace("\t", "").replace("\n", "")

        # telefone
        pattern = r'Fone:(.+)'
        telefone = find_pattern(pattern, text)
        telefone[0] = telefone[0].replace(" ", "").replace("\t", "").replace("\n", "")

        # email
        pattern = "FOB\n\n(.*)"
        email = find_pattern(pattern, text)
 
        # endereco
        pattern = r'Comprador\(a\):\n\n.+\n\n(.+)'
        endereco = find_pattern(pattern, text)

        # descricao
        pattern =r"Gar\. Mês(.*?)Peso"
        descricao = find_largepattern(pattern, text)
        
        # Create DataFrame
        data = {
            'Arquivo': arquivo,
            'Recibo': recibo,
            'Ordem de Servico': oss,
            'Data': data_entrada,
            'Cliente': nome_cliente,
            'CPF/CNPJ': cpf_cnpj,
            'Telefone': telefone,
            'Email': email,
            'Endereco': endereco,
            'Descrição': descricao
        }

        info_word = pd.DataFrame(data)
        
        info_word['Descrição'] = info_word['Descrição'].str.replace("\n\n", " ")
        info_word['Email'] = info_word['Email'].str.strip(" ")
       
        full_data = pd.concat([full_data, info_word])


In [ ]:
full_data.to_csv("mail_list.csv", encoding=("utf-8-sig"))